In [58]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

In [59]:
input = pd.read_csv('artists.csv', header = None)
artist_ids = []
for link in input[0]:
    
    artist_ids.append(link[32:54])

artist_ids

['6eJa3zG1QZLRB3xgRuyxbm', '4j5KBTO4tk7up54ZirNGvK']

In [69]:
albums = []
album_track_ids = []

for artist_id in artist_ids:
    
    album_ids = []
    artist_uri = 'spotify:artist:' + artist_id

    # Add album ids to album_ids list
    results = sp.artist_albums(artist_uri, album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    for album in albums:
        album_ids.append(album['id'])

all_tracks = []
for id in album_ids:

    track_ids = []

    results = sp.album_tracks(id)
    tracks = results['items']
    all_tracks.extend(tracks)
    for track in tracks:
        track_ids.append(track['id'])

    album_track_ids.append(track_ids)


assert len(album_track_ids) == len(albums)

len(albums)

6

In [77]:
all_tracks[0]['track_number']


1

In [79]:
columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['danceability',
'energy',
'key',
'loudness',
'mode',
'speechiness',
'acousticness',
'instrumentalness',
'liveness',
'valence',
'tempo',
'id',
'duration_ms',
'time_signature',
'track_title',
'album_title',
'album_artist',
'track_number',
'total_tracks']
data = pd.DataFrame(columns=headings)

# Create DataFrames of features from track ids
i = 0
j = 0
for album in album_track_ids:
    
    length = len(album)
    
    
    for track in album:
        results = sp.audio_features(track)
        features = results[0]
        features_matrix = pd.DataFrame.from_records(features, index=[0])

        # Remove unneeded columns
        features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)
        # print(features_matrix)

        # Add track title
        track_title = all_tracks[j]['name']
        features_matrix['track_title'] = track_title
        # Add album title
        album_title = albums[i]['name']
        features_matrix['album_title'] = album_title
        # Add artist name
        name = albums[i]['artists'][0]['name']
        features_matrix['album_artist'] = name

        # Add track number and total tracks
        features_matrix['track_number'] = all_tracks[j]['track_number']
        features_matrix['total_tracks'] = length

        # Add album as a row to the data df
        # print(features_matrix,'\n')
        # print(data)
        data = data.append(features_matrix)

        j += 1

    i += 1

data

data.drop_duplicates(subset=['track_number', 'album_title', 'album_artist'])

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,track_title,album_title,album_artist,track_number,total_tracks
0,0.817,0.661,0,-4.835,0,0.1020,0.066800,0.000003,0.1620,0.9450,139.994,3CzfTl1RSsQ6jnxKZmsJMe,114320,4,Something,Harmony House,Dayglow,1,11
0,0.866,0.754,0,-5.531,1,0.0356,0.036400,0.001810,0.0801,0.9620,115.034,0CWvHfEwbeBVN0RViTybvL,229173,4,Medicine,Harmony House,Dayglow,2,11
0,0.702,0.827,11,-5.588,0,0.0778,0.000381,0.568000,0.0333,0.9770,150.019,3W0fmsdPgPisEQEfefimGT,151267,4,Balcony,Harmony House,Dayglow,3,11
0,0.756,0.517,7,-8.044,1,0.0883,0.197000,0.000118,0.0767,0.4230,79.997,4oB6hNxskthsaZB1OuxgZX,191747,4,December,Harmony House,Dayglow,4,11
0,0.875,0.649,6,-5.754,1,0.0433,0.429000,0.002370,0.0704,0.8330,140.031,0brBOx8ejPXvBYnQkWquYK,194693,4,Close To You,Harmony House,Dayglow,5,11
0,0.607,0.617,9,-8.236,1,0.0257,0.001640,0.535000,0.1460,0.2720,99.998,76rMrk4t6Cmg2EXR7VV8xs,263080,4,Crying on the Dancefloor,Harmony House,Dayglow,6,11
0,0.700,0.493,6,-8.880,1,0.0277,0.025800,0.097300,0.0799,0.3980,134.994,3R12JORSoLUDpSzpS8semL,219800,4,Into Blue,Harmony House,Dayglow,7,11
0,0.894,0.574,7,-8.206,1,0.0528,0.021400,0.000025,0.0475,0.4350,119.988,1c8g7AOUIdREi458yJNIdq,180827,4,Moving Out,Harmony House,Dayglow,8,11
0,0.497,0.675,2,-7.442,1,0.0332,0.003500,0.000367,0.2190,0.3380,79.965,0PGnwDPPMoJ1s49hQQXHFR,208587,4,Woah Man,Harmony House,Dayglow,9,11
0,0.477,0.440,11,-8.771,1,0.0267,0.155000,0.000854,0.0636,0.1660,72.536,0iN2lUhuFEjQ5hHGXDO7ET,172893,4,Strangers,Harmony House,Dayglow,10,11


In [82]:
data['label'] = data['track_number']/data['total_tracks']

# data.to_csv('./data/data.csv')
data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,track_title,album_title,album_artist,track_number,total_tracks,label
0,0.817,0.661,0,-4.835,0,0.1020,0.066800,0.000003,0.1620,0.945,139.994,3CzfTl1RSsQ6jnxKZmsJMe,114320,4,Something,Harmony House,Dayglow,1,11,0.090909
0,0.866,0.754,0,-5.531,1,0.0356,0.036400,0.001810,0.0801,0.962,115.034,0CWvHfEwbeBVN0RViTybvL,229173,4,Medicine,Harmony House,Dayglow,2,11,0.181818
0,0.702,0.827,11,-5.588,0,0.0778,0.000381,0.568000,0.0333,0.977,150.019,3W0fmsdPgPisEQEfefimGT,151267,4,Balcony,Harmony House,Dayglow,3,11,0.272727
0,0.756,0.517,7,-8.044,1,0.0883,0.197000,0.000118,0.0767,0.423,79.997,4oB6hNxskthsaZB1OuxgZX,191747,4,December,Harmony House,Dayglow,4,11,0.363636
0,0.875,0.649,6,-5.754,1,0.0433,0.429000,0.002370,0.0704,0.833,140.031,0brBOx8ejPXvBYnQkWquYK,194693,4,Close To You,Harmony House,Dayglow,5,11,0.454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.536,0.883,8,-2.022,0,0.0530,0.119000,0.000692,0.7030,0.439,128.018,43ReOrkVrq0ewq02sf9X0c,213882,4,Reason,Collateral,NERVO,11,15,0.733333
0,0.624,0.756,1,-7.033,1,0.0579,0.132000,0.000028,0.2880,0.586,128.044,7oXaDhWwGOhklKGzpCWARg,187500,4,Right Thru Me,Collateral,NERVO,12,15,0.8
0,0.439,0.894,1,-5.191,0,0.0698,0.010900,0.017700,0.1350,0.288,125.961,6CBc5nUXYftNQpRb5IH7KL,201307,4,You’re Gonna Love Again,Collateral,NERVO,13,15,0.866667
0,0.539,0.869,0,-4.442,0,0.0439,0.000054,0.064300,0.2090,0.402,126.983,0mEFIz5a0ONVFWCrgz90WA,341253,4,We're All No One - NERVO Goes To Paris Remix,Collateral,NERVO,14,15,0.933333


In [16]:
# Live
sp.audio_features('2RpXUwCytDFqZSHl6kIquB')

[{'danceability': 0.26,
  'energy': 0.99,
  'key': 7,
  'loudness': -3.498,
  'mode': 1,
  'speechiness': 0.128,
  'acousticness': 0.00202,
  'instrumentalness': 0.00416,
  'liveness': 0.779,
  'valence': 0.0885,
  'tempo': 128.29,
  'type': 'audio_features',
  'id': '2RpXUwCytDFqZSHl6kIquB',
  'uri': 'spotify:track:2RpXUwCytDFqZSHl6kIquB',
  'track_href': 'https://api.spotify.com/v1/tracks/2RpXUwCytDFqZSHl6kIquB',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2RpXUwCytDFqZSHl6kIquB',
  'duration_ms': 349733,
  'time_signature': 4}]

In [103]:
# Remastered
sp.audio_features('0xzhryP1AoHUazYdJ5rj3B')

[{'danceability': 0.46,
  'energy': 0.922,
  'key': 2,
  'loudness': -7.388,
  'mode': 1,
  'speechiness': 0.264,
  'acousticness': 0.0449,
  'instrumentalness': 0.000252,
  'liveness': 0.34,
  'valence': 0.328,
  'tempo': 119.961,
  'type': 'audio_features',
  'id': '0xzhryP1AoHUazYdJ5rj3B',
  'uri': 'spotify:track:0xzhryP1AoHUazYdJ5rj3B',
  'track_href': 'https://api.spotify.com/v1/tracks/0xzhryP1AoHUazYdJ5rj3B',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0xzhryP1AoHUazYdJ5rj3B',
  'duration_ms': 311027,
  'time_signature': 4}]

In [104]:
# Non-remastered (original)
sp.audio_features('4BB3Itr3YETOKFURnC28Ld')

[{'danceability': 0.472,
  'energy': 0.859,
  'key': 2,
  'loudness': -11.866,
  'mode': 1,
  'speechiness': 0.181,
  'acousticness': 0.0793,
  'instrumentalness': 0.000202,
  'liveness': 0.645,
  'valence': 0.322,
  'tempo': 119.989,
  'type': 'audio_features',
  'id': '4BB3Itr3YETOKFURnC28Ld',
  'uri': 'spotify:track:4BB3Itr3YETOKFURnC28Ld',
  'track_href': 'https://api.spotify.com/v1/tracks/4BB3Itr3YETOKFURnC28Ld',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4BB3Itr3YETOKFURnC28Ld',
  'duration_ms': 310667,
  'time_signature': 4}]